In [3]:
import glob

In [30]:
import traceback
import datetime

In [2]:
# presto-test.py
# simple python script to run a query on a presto cluster and display the result set 
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
engine = create_engine("presto://localhost:8080/ahana_hive", connect_args={'protocol': 'http'})
 
with engine.connect() as con:
    rs = con.execute('SELECT now()')
    for row in rs:
        print(row)

('2021-10-21 02:18:16.614 UTC',)


In [1]:
!pwd

/home/wengong/projects/py4kids/lesson-14-db/presto


In [4]:
sql_files = glob.glob("./benchmark/tpcds/*.sql")

In [5]:
len(sql_files)

103

In [7]:
sorted(sql_files)

['./benchmark/tpcds/q01.sql',
 './benchmark/tpcds/q02.sql',
 './benchmark/tpcds/q03.sql',
 './benchmark/tpcds/q04.sql',
 './benchmark/tpcds/q05.sql',
 './benchmark/tpcds/q06.sql',
 './benchmark/tpcds/q07.sql',
 './benchmark/tpcds/q08.sql',
 './benchmark/tpcds/q09.sql',
 './benchmark/tpcds/q10.sql',
 './benchmark/tpcds/q11.sql',
 './benchmark/tpcds/q12.sql',
 './benchmark/tpcds/q13.sql',
 './benchmark/tpcds/q14_1.sql',
 './benchmark/tpcds/q14_2.sql',
 './benchmark/tpcds/q15.sql',
 './benchmark/tpcds/q16.sql',
 './benchmark/tpcds/q17.sql',
 './benchmark/tpcds/q18.sql',
 './benchmark/tpcds/q19.sql',
 './benchmark/tpcds/q20.sql',
 './benchmark/tpcds/q21.sql',
 './benchmark/tpcds/q22.sql',
 './benchmark/tpcds/q23_1.sql',
 './benchmark/tpcds/q23_2.sql',
 './benchmark/tpcds/q24_1.sql',
 './benchmark/tpcds/q24_2.sql',
 './benchmark/tpcds/q25.sql',
 './benchmark/tpcds/q26.sql',
 './benchmark/tpcds/q27.sql',
 './benchmark/tpcds/q28.sql',
 './benchmark/tpcds/q29.sql',
 './benchmark/tpcds/q30.sql'

In [18]:
database,schema,table = "tpcds.tiny.customer".split(".")

In [19]:
database,schema,table

('tpcds', 'tiny', 'customer')

In [20]:
sql = f"""
describe {database}.{schema}.{table}
"""

columns = []
with engine.connect() as con:
    rs = con.execute(sql)
    for row in rs:
        columns.append(row)

In [21]:
len(columns), columns

(18,
 [('c_customer_sk', 'bigint', '', ''),
  ('c_customer_id', 'char(16)', '', ''),
  ('c_current_cdemo_sk', 'bigint', '', ''),
  ('c_current_hdemo_sk', 'bigint', '', ''),
  ('c_current_addr_sk', 'bigint', '', ''),
  ('c_first_shipto_date_sk', 'bigint', '', ''),
  ('c_first_sales_date_sk', 'bigint', '', ''),
  ('c_salutation', 'char(10)', '', ''),
  ('c_first_name', 'char(20)', '', ''),
  ('c_last_name', 'char(30)', '', ''),
  ('c_preferred_cust_flag', 'char(1)', '', ''),
  ('c_birth_day', 'integer', '', ''),
  ('c_birth_month', 'integer', '', ''),
  ('c_birth_year', 'integer', '', ''),
  ('c_birth_country', 'varchar(20)', '', ''),
  ('c_login', 'char(13)', '', ''),
  ('c_email_address', 'char(50)', '', ''),
  ('c_last_review_date_sk', 'bigint', '', '')])

In [15]:
sql = f"""
select * from {catalog}.{schema}.{table} limit 10
"""

results = []
with engine.connect() as con:
    rs = con.execute(sql)
    for row in rs:
        results.append(row)

In [16]:
len(results)

10

In [22]:
len(results[0])

18

In [24]:
sql = "select * from ${database}.${schema}.customer limit 2"

In [25]:
sql.replace("$","").format(database=database,schema=schema)

'select * from tpcds.tiny.customer limit 2'

In [31]:
conn = engine.connect()

In [35]:
i = 0
for file_sql in sorted(sql_files):
    # if i > 1: break
        
    print(f"\n\nSQL name: {file_sql}...\n========================================")
    sql = open(file_sql).read().replace("$","").format(database=database,schema=schema)
    print(sql)
    print("\n")
    
    results = []
    start = datetime.datetime.now()
    try:
        rs = conn.execute(sql)
        stop = datetime.datetime.now()
        for row in rs:
            results.append(row)
            
        print(f"Results: completed in {(stop - start).seconds} sec\n+++++++++++++++++++++++++++++++++\n{results[:5]}")
    except Exception:
        print(traceback.format_exc())
        
    i += 1
            



SQL name: ./benchmark/tpcds/q01.sql...
WITH
  customer_total_return AS (
   SELECT
     "sr_customer_sk" "ctr_customer_sk"
   , "sr_store_sk" "ctr_store_sk"
   , "sum"("sr_return_amt") "ctr_total_return"
   FROM
     tpcds.tiny.store_returns
   , tpcds.tiny.date_dim
   WHERE ("sr_returned_date_sk" = "d_date_sk")
      AND ("d_year" = 2000)
   GROUP BY "sr_customer_sk", "sr_store_sk"
) 
SELECT "c_customer_id"
FROM
  customer_total_return ctr1
, tpcds.tiny.store
, tpcds.tiny.customer
WHERE ("ctr1"."ctr_total_return" > (
      SELECT ("avg"("ctr_total_return") * DECIMAL '1.2')
      FROM
        customer_total_return ctr2
      WHERE ("ctr1"."ctr_store_sk" = "ctr2"."ctr_store_sk")
   ))
   AND ("s_store_sk" = "ctr1"."ctr_store_sk")
   AND ("s_state" = 'TN')
   AND ("ctr1"."ctr_customer_sk" = "c_customer_sk")
ORDER BY "c_customer_id" ASC
LIMIT 100



Results: completed in 1 sec
+++++++++++++++++++++++++++++++++
[('AAAAAAAAAACAAAAA',), ('AAAAAAAAAADAAAAA',), ('AAAAAAAAACCAAAAA',), ('AAAAA

Results: completed in 2 sec
+++++++++++++++++++++++++++++++++
[('AAAAAAAADOCAAAAA', 'Joy                 ', 'Baker                         ', 'Y'), ('AAAAAAAAHMBAAAAA', 'Eunice              ', 'Shaw                          ', 'N'), ('AAAAAAAAIICAAAAA', 'Sherri              ', 'Tyler                         ', 'Y'), ('AAAAAAAAJCAAAAAA', 'Maxine              ', 'Carlson                       ', 'N'), ('AAAAAAAAJGBAAAAA', 'Bonnie              ', 'Harrison                      ', 'Y')]


SQL name: ./benchmark/tpcds/q05.sql...
WITH
  ssr AS (
   SELECT
     "s_store_id"
   , "sum"("sales_price") "sales"
   , "sum"("profit") "profit"
   , "sum"("return_amt") "returns"
   , "sum"("net_loss") "profit_loss"
   FROM
     (
      SELECT
        "ss_store_sk" "store_sk"
      , "ss_sold_date_sk" "date_sk"
      , "ss_ext_sales_price" "sales_price"
      , "ss_net_profit" "profit"
      , CAST(0 AS DECIMAL(7,2)) "return_amt"
      , CAST(0 AS DECIMAL(7,2)) "net_loss"
      FROM
        tpcds.tiny.

Results: completed in 2 sec
+++++++++++++++++++++++++++++++++
[]


SQL name: ./benchmark/tpcds/q09.sql...
SELECT
  (CASE WHEN ((
      SELECT "count"(*)
      FROM
        tpcds.tiny.store_sales
      WHERE ("ss_quantity" BETWEEN 1 AND 20)
   ) > 74129) THEN (
   SELECT "avg"("ss_ext_discount_amt")
   FROM
     tpcds.tiny.store_sales
   WHERE ("ss_quantity" BETWEEN 1 AND 20)
) ELSE (
   SELECT "avg"("ss_net_paid")
   FROM
     tpcds.tiny.store_sales
   WHERE ("ss_quantity" BETWEEN 1 AND 20)
) END) "bucket1"
, (CASE WHEN ((
      SELECT "count"(*)
      FROM
        tpcds.tiny.store_sales
      WHERE ("ss_quantity" BETWEEN 21 AND 40)
   ) > 122840) THEN (
   SELECT "avg"("ss_ext_discount_amt")
   FROM
     tpcds.tiny.store_sales
   WHERE ("ss_quantity" BETWEEN 21 AND 40)
) ELSE (
   SELECT "avg"("ss_net_paid")
   FROM
     tpcds.tiny.store_sales
   WHERE ("ss_quantity" BETWEEN 21 AND 40)
) END) "bucket2"
, (CASE WHEN ((
      SELECT "count"(*)
      FROM
        tpcds.tiny.store_sales
 

Results: completed in 2 sec
+++++++++++++++++++++++++++++++++
[('AAAAAAAABMDAAAAA', 'Jose                ', 'Guzman                        ', 'N', 'MARSHALL ISLANDS', None), ('AAAAAAAAFLAAAAAA', 'Joseph              ', 'Riley                         ', 'N', 'BELIZE', None), ('AAAAAAAAFNCAAAAA', 'Jennifer            ', 'Fitzpatrick                   ', 'N', 'UNITED ARAB EMIRATES', None), ('AAAAAAAAJCAAAAAA', 'Maxine              ', 'Carlson                       ', 'N', 'ISLE OF MAN', None), ('AAAAAAAAJGBAAAAA', 'Bonnie              ', 'Harrison                      ', 'Y', 'EL SALVADOR', None)]


SQL name: ./benchmark/tpcds/q12.sql...
SELECT
  "i_item_id"
, "i_item_desc"
, "i_category"
, "i_class"
, "i_current_price"
, "sum"("ws_ext_sales_price") "tpcds.tiny.itemrevenue"
, (("sum"("ws_ext_sales_price") * 100) / "sum"("sum"("ws_ext_sales_price")) OVER (PARTITION BY "i_class")) "revenueratio"
FROM
  tpcds.tiny.web_sales
, tpcds.tiny.item
, tpcds.tiny.date_dim
WHERE ("ws_item_sk" = "i_ite

Traceback (most recent call last):
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/sqlalchemy/engine/cursor.py", line 955, in fetchone
    row = dbapi_cursor.fetchone()
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/common.py", line 106, in fetchone
    self._fetch_while(lambda: not self._data and self._state != self._STATE_FINISHED)
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/common.py", line 46, in _fetch_while
    self._fetch_more()
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/presto.py", line 308, in _fetch_more
    self._process_response(self._requests_session.get(self._nextUri, **self._requests_kwargs))
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/presto.py", line 349, in _process_response
    raise DatabaseError(response_json['error'])
pyhive.exc.DatabaseError: {'message': 'Query exceeded per-node user memory limit of 102.40MB [Allo

...

Traceback (most recent call last):
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/sqlalchemy/engine/cursor.py", line 955, in fetchone
    row = dbapi_cursor.fetchone()
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/common.py", line 106, in fetchone
    self._fetch_while(lambda: not self._data and self._state != self._STATE_FINISHED)
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/common.py", line 46, in _fetch_while
    self._fetch_more()
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/presto.py", line 308, in _fetch_more
    self._process_response(self._requests_session.get(self._nextUri, **self._requests_kwargs))
  File "/home/wengong/anaconda3/envs/py39/lib/python3.9/site-packages/pyhive/presto.py", line 349, in _process_response
    raise DatabaseError(response_json['error'])
pyhive.exc.DatabaseError: {'message': 'Query exceeded per-node user memory limit of 102.40MB [Allo

Results: completed in 1 sec
+++++++++++++++++++++++++++++++++
[('5189585.33', None, None, 2, 1), ('655046.32', 'Men                                               ', None, 1, 1), ('626624.82', 'Women                                             ', None, 1, 2), ('581067.68', 'Books                                             ', None, 1, 3), ('561790.19', 'Music                                             ', None, 1, 4)]


SQL name: ./benchmark/tpcds/q87.sql...
SELECT "count"(*)
FROM
  (
(
      SELECT DISTINCT
        "c_last_name"
      , "c_first_name"
      , "d_date"
      FROM
        tpcds.tiny.store_sales
      , tpcds.tiny.date_dim
      , tpcds.tiny.customer
      WHERE ("store_sales"."ss_sold_date_sk" = "date_dim"."d_date_sk")
         AND ("store_sales"."ss_customer_sk" = "customer"."c_customer_sk")
         AND ("d_month_seq" BETWEEN 1200 AND (1200 + 11))
   ) EXCEPT (
      SELECT DISTINCT
        "c_last_name"
      , "c_first_name"
      , "d_date"
      FROM
        tpcds.

Results: completed in 2 sec
+++++++++++++++++++++++++++++++++
[(0, 0, 0, 0, 0, 0, 0, 0)]


SQL name: ./benchmark/tpcds/q89.sql...
SELECT *
FROM
  (
   SELECT
     "i_category"
   , "i_class"
   , "i_brand"
   , "s_store_name"
   , "s_company_name"
   , "d_moy"
   , "sum"("ss_sales_price") "sum_sales"
   , "avg"("sum"("ss_sales_price")) OVER (PARTITION BY "i_category", "i_brand", "s_store_name", "s_company_name") "avg_monthly_sales"
   FROM
     tpcds.tiny.item
   , tpcds.tiny.store_sales
   , tpcds.tiny.date_dim
   , tpcds.tiny.store
   WHERE ("ss_item_sk" = "i_item_sk")
      AND ("ss_sold_date_sk" = "d_date_sk")
      AND ("ss_store_sk" = "s_store_sk")
      AND ("d_year" IN (1999))
      AND ((("i_category" IN ('Books'         , 'Electronics'         , 'Sports'))
            AND ("i_class" IN ('computers'         , 'stereo'         , 'football')))
         OR (("i_category" IN ('Men'         , 'Jewelry'         , 'Women'))
            AND ("i_class" IN ('shirts'         , 'birdal'  

Results: completed in 2 sec
+++++++++++++++++++++++++++++++++
[]


SQL name: ./benchmark/tpcds/q92.sql...
SELECT "sum"("ws_ext_discount_amt") "Excess Discount Amount"
FROM
  tpcds.tiny.web_sales
, tpcds.tiny.item
, tpcds.tiny.date_dim
WHERE ("i_manufact_id" = 350)
   AND ("i_item_sk" = "ws_item_sk")
   AND ("d_date" BETWEEN CAST('2000-01-27' AS DATE) AND (CAST('2000-01-27' AS DATE) + INTERVAL  '90' DAY))
   AND ("d_date_sk" = "ws_sold_date_sk")
   AND ("ws_ext_discount_amt" > (
      SELECT (DECIMAL '1.3' * "avg"("ws_ext_discount_amt"))
      FROM
        tpcds.tiny.web_sales
      , tpcds.tiny.date_dim
      WHERE ("ws_item_sk" = "i_item_sk")
         AND ("d_date" BETWEEN CAST('2000-01-27' AS DATE) AND (CAST('2000-01-27' AS DATE) + INTERVAL  '90' DAY))
         AND ("d_date_sk" = "ws_sold_date_sk")
   ))
ORDER BY "sum"("ws_ext_discount_amt") ASC
LIMIT 100



Results: completed in 1 sec
+++++++++++++++++++++++++++++++++
[(None,)]


SQL name: ./benchmark/tpcds/q93.sql...
SELECT
  "ss_c

Results: completed in 2 sec
+++++++++++++++++++++++++++++++++
[('Conventional childr', 'EXPRESS                       ', 'Mid Atlantic', 520, 567, 562, 0, 0), ('Conventional childr', 'EXPRESS                       ', 'NY Metro', 590, 679, 571, 0, 0), ('Conventional childr', 'LIBRARY                       ', 'Mid Atlantic', 378, 420, 457, 0, 0), ('Conventional childr', 'LIBRARY                       ', 'NY Metro', 426, 474, 442, 0, 0), ('Conventional childr', 'NEXT DAY                      ', 'Mid Atlantic', 544, 570, 581, 0, 0)]
